# First Trial
- Player0 = Random Choice
- Player1 = Always Cooperate
- Player2 = Always Exploit

In [1]:
import random

def player0():
    """Player0 randomly chooses to Cooperate (CO) or Exploit (EX)."""
    return random.choice(["CO", "EX"])

def player1():
    """Player1 always chooses to Cooperate (CO)."""
    return "CO"

def player2():
    """Player2 always chooses to Exploit (EX)."""
    return "EX"


In [2]:
class Game:

    def __init__(self):
        self.scores = {"P1": 0, "P2": 0}
        self.history = []

    def play_round(self, choice1, choice2):
        if choice1 == "CO" and choice2 == "CO":
            self.scores["P1"] += 1
            self.scores["P2"] += 1
        elif choice1 == "CO" and choice2 == "EX":
            self.scores["P1"] += 0
            self.scores["P2"] += 5
        elif choice1 == "EX" and choice2 == "CO":
            self.scores["P1"] += 5
            self.scores["P2"] += 0
        elif choice1 == "EX" and choice2 == "EX":
            self.scores["P1"] += 0
            self.scores["P2"] += 0

    def play_game(self, rounds, strategy1, strategy2):
        self.scores = {"P1": 0, "P2": 0}  # Reset scores for each game
        for _ in range(rounds):
            choice1 = strategy1()
            choice2 = strategy2()
            self.play_round(choice1, choice2)
        return self.scores

    def get_scores(self):
        return self.scores


In [3]:
def tournament(players, tournament_rounds, rounds_per_match):
    total_scores = {f"Player{i}": 0 for i in range(len(players))}
    
    for i in range(len(players)):
        for j in range(i+1, len(players)):
            for _ in range(tournament_rounds):
                game = Game()
                scores = game.play_game(rounds_per_match, players[i], players[j])
                total_scores[f"Player{i}"] += scores["P1"]
                total_scores[f"Player{j}"] += scores["P2"]

    return total_scores


In [4]:
players = [player0, player1, player2]
tournament_rounds = 3
rounds_per_match = 10

final_scores = tournament(players, tournament_rounds, rounds_per_match)
for player, score in final_scores.items():
    print(f"{player} = {score} points")

Player0 = 74 points
Player1 = 19 points
Player2 = 240 points


# Second Trial
- Player0 = Random Choice
- Player1 = Always Cooperate
- Player2 = Always Exploit
- Player3 = Tit-for-Tat: It always starts with Cooperate. But in round 2 forwards, it will mimic exactly what the opponent did. 

In [5]:
class Game:
    def __init__(self, player1, player2):
        self.scores = {"P1": 0, "P2": 0}
        self.player1 = player1
        self.player2 = player2


    def play_round(self, choice1, choice2):
        choice1 = self.player1() if callable(self.player1) else self.player1.play()
        choice2 = self.player2() if callable(self.player2) else self.player2.play()

        if choice1 == "CO" and choice2 == "CO":
            self.scores["P1"] += 3
            self.scores["P2"] += 3
        elif choice1 == "CO" and choice2 == "EX":
            self.scores["P1"] += 0
            self.scores["P2"] += 5
        elif choice1 == "EX" and choice2 == "CO":
            self.scores["P1"] += 5
            self.scores["P2"] += 0
        elif choice1 == "EX" and choice2 == "EX":
            self.scores["P1"] += 1
            self.scores["P2"] += 1
    
        # Record the choices and scores for this round
        self.history.append({
            "P1_choice": choice1,
            "P2_choice": choice2,
            "Scores": self.scores.copy()
        })

        if hasattr(self.player1, 'update_last_move'):
            self.player1.update_last_move(choice2)
        if hasattr(self.player2, 'update_last_move'):
            self.player2.update_last_move(choice1)

    def play_game(self, rounds):
        self.scores = {"P1": 0, "P2": 0}  # Reset scores for each game
        for _ in range(rounds):
            self.play_round()
        return self.scores

    def get_scores(self):
        return self.scores

    def get_history(self):
        return self.history

In [6]:
def tournament(players, tournament_rounds, rounds_per_match):
    total_scores = {f"Player{i}": 0 for i in range(len(players))}
    
    # Create instances for stateful players
    player_instances = [p() if callable(p) else p for p in players]

    for i, player1 in enumerate(player_instances):
        for j, player2 in enumerate(player_instances):
            if i < j:
                for _ in range(tournament_rounds):
                    game = Game(player1, player2)
                    scores = game.play_game(rounds_per_match)
                    total_scores[f"Player{i}"] += scores["P1"]
                    total_scores[f"Player{j}"] += scores["P2"]

    # Sort and return scores
    sorted_scores = sorted(total_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_scores

In [7]:
import random

def player0():
    """Player0 randomly chooses to Cooperate (CO) or Exploit (EX)."""
    return random.choice(["CO", "EX"])

def player1():
    """Player1 always chooses to Cooperate (CO)."""
    return "CO"

def player2():
    """Player2 always chooses to Exploit (EX)."""
    return "EX"


class Player3:
    def __init__(self):
        self.last_move = None

    def play(self):
        if self.last_move is None:
            return "CO"
        return self.last_move

    def update_last_move(self, opponent_move):
        self.last_move = opponent_move

In [8]:
player3 = Player3()  # Initialize Player3 as an instance
player3a = Player3()
players = [player0, player1, player2, player3, player3a]

tournament_rounds = 3
rounds_per_match = 10

final_scores = tournament(players, tournament_rounds, rounds_per_match)
for player, score in final_scores:
    print(f"{player} = {score} points")

TypeError: Game.play_round() missing 2 required positional arguments: 'choice1' and 'choice2'